# Trabalho 2 da disciplina de APL
Integrantes do grupo:
* Daniel Brai Gonzales Marcos
* Giovanna Nascimento Antonieti
* Priscila Marques de Oliveira

In [1]:
import tensorflow as tf
from tensorflow.keras.utils import Sequence
import numpy as np
import matplotlib.pyplot as plt
import math
import cv2

Init Plugin
Init Graph Optimizer
Init Kernel


In [2]:
tf.__version__

'2.5.0'

In [3]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [4]:
#gpus = tf.config.list_physical_devices('GPU')

#if gpus:
 #   try:
        # Currently, memory growth needs to be the same across GPUs
  #      for gpu in gpus:
   #         tf.config.experimental.set_memory_growth(gpu, True)
    #    logical_gpus = tf.config.list_logical_devices('GPU')
     #   print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
   # except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
    #    print(e)

1Metal device set to: Apple M1
 Physical GPUs, 1 Logical GPUs


2022-06-15 09:11:43.791148: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-06-15 09:11:43.791276: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [4]:
import pandas as pd
import numpy as np
import os

# 1. Dataset 
* Load
* Labels
* Pré-processamento 

In [5]:
# train
dataset_df_train = pd.read_csv('./dataset/train.csv')

# test
dataset_df_test = pd.read_csv('./dataset/test.csv')
dataset_folder ='./dataset/imgs'
class_folders = sorted(os.listdir(dataset_folder))
n_samples=[]
print(class_folders)
print(f'Number of class: {len(class_folders)}')

for class_folder in class_folders:
    full_class_folder = os.path.join(dataset_folder, class_folder)
    
    class_img_filenames = os.listdir(full_class_folder)
    n_samples.append(len(class_img_filenames))
    print(f'Number of Images for Class "{class_folder}": {len(class_img_filenames)}')

['Adam Sandler', 'Alec Baldwin', 'Angelina Jolie', 'Anna Kournikova', 'Ashton Kutcher', 'Avril Lavigne', 'Barack Obama', 'Ben Affleck', 'Beyonce Knowles', 'Brad Pitt', 'Cameron Diaz', 'Cate Blanchett', 'Charlize Theron', 'Christina Ricci', 'Claudia Schiffer', 'Clive Owen', 'Colin Farrell', 'Colin Powell', 'Cristiano Ronaldo', 'Daniel Craig', 'Daniel Radcliffe', 'David Beckham', 'David Duchovny', 'Denise Richards', 'Drew Barrymore', 'Dustin Hoffman', 'Ehud Olmert', 'Eva Mendes', 'Faith Hill', 'George Clooney', 'Gordon Brown', 'Gwyneth Paltrow', 'Halle Berry', 'Harrison Ford', 'Hugh Jackman', 'Hugh Laurie', 'Jack Nicholson', 'Jennifer Aniston', 'Jennifer Lopez', 'Jennifer Love Hewitt', 'Jessica Alba', 'Jessica Simpson', 'Joaquin Phoenix', 'John Travolta', 'Julia Roberts', 'Julia Stiles', 'Kate Moss', 'Kate Winslet', 'Katherine Heigl', 'Keira Knightley', 'Kiefer Sutherland', 'Leonardo DiCaprio', 'Lindsay Lohan', 'Mariah Carey', 'Martha Stewart', 'Matt Damon', 'Meg Ryan', 'Meryl Streep', '

In [32]:
print(f'Mínimo: {min(n_samples)}')
print(f'Máximo: {max(n_samples)}')
n_samples=max(n_samples)

Mínimo: 100
Máximo: 367


In [8]:
import random

img_full_paths = []
img_classes = []

# OPTIONAL: just to get the same selected images
random.seed(42)

for class_folder in sorted(os.listdir(dataset_folder)):
    print(f'Class: {class_folder}') 
    
    # translated class
    img_class = class_folder
    
    # get the full class folder pathname
    full_class_folder = os.path.join(dataset_folder, class_folder)
    
    # get all image filenames (without their parent dir) for the current class/animal
    class_img_filenames = sorted(os.listdir(full_class_folder))
    
    
    # undersampling from scratch
    random.shuffle(class_img_filenames)
    class_img_filenames = class_img_filenames[:]
    print(f'Number of images: {len(class_img_filenames)}')
    
    for img_filename in class_img_filenames:
        full_img_path = os.path.join(full_class_folder, img_filename)
        
        img_full_paths.append(full_img_path)
        img_classes.append(img_class)
    
    print()

Class: Adam Sandler
Number of images: 108

Class: Alec Baldwin
Number of images: 103

Class: Angelina Jolie
Number of images: 214

Class: Anna Kournikova
Number of images: 171

Class: Ashton Kutcher
Number of images: 101

Class: Avril Lavigne
Number of images: 299

Class: Barack Obama
Number of images: 268

Class: Ben Affleck
Number of images: 117

Class: Beyonce Knowles
Number of images: 126

Class: Brad Pitt
Number of images: 300

Class: Cameron Diaz
Number of images: 246

Class: Cate Blanchett
Number of images: 160

Class: Charlize Theron
Number of images: 195

Class: Christina Ricci
Number of images: 143

Class: Claudia Schiffer
Number of images: 122

Class: Clive Owen
Number of images: 134

Class: Colin Farrell
Number of images: 145

Class: Colin Powell
Number of images: 112

Class: Cristiano Ronaldo
Number of images: 168

Class: Daniel Craig
Number of images: 168

Class: Daniel Radcliffe
Number of images: 246

Class: David Beckham
Number of images: 187

Class: David Duchovny
Numb

In [9]:
import pandas as pd

dataset_df = pd.DataFrame({
    'image_pathname': img_full_paths,
    'class': img_classes
})

dataset_df

,image_pathname,class
0,./dataset/imgs/Adam Sandler/277.jpg,Adam Sandler
1,./dataset/imgs/Adam Sandler/134.jpg,Adam Sandler
2,./dataset/imgs/Adam Sandler/108.jpg,Adam Sandler
3,./dataset/imgs/Adam Sandler/3.jpg,Adam Sandler
4,./dataset/imgs/Adam Sandler/84.jpg,Adam Sandler
...,...,...
13833,./dataset/imgs/Zac Efron/461.jpg,Zac Efron
13834,./dataset/imgs/Zac Efron/326.jpg,Zac Efron
13835,./dataset/imgs/Zac Efron/393.jpg,Zac Efron
13836,./dataset/imgs/Zac Efron/138.jpg,Zac Efron


In [10]:
imgs = []
classes = []
for i in dataset_df_train['image_pathname']:
    data = dataset_df.loc[dataset_df['image_pathname'] == './dataset/'+i]
    imgs.append(data.iat[0,0])
    classes.append(data.iat[0,1])
    
df_train = pd.DataFrame({
    'image_pathname': imgs,
    'class': classes
})

df_train

,image_pathname,class
0,./dataset/imgs/Adam Sandler/277.jpg,Adam Sandler
1,./dataset/imgs/Adam Sandler/136.jpg,Adam Sandler
2,./dataset/imgs/Adam Sandler/198.jpg,Adam Sandler
3,./dataset/imgs/Adam Sandler/234.jpg,Adam Sandler
4,./dataset/imgs/Adam Sandler/202.jpg,Adam Sandler
...,...,...
12173,./dataset/imgs/Zac Efron/120.jpg,Zac Efron
12174,./dataset/imgs/Zac Efron/476.jpg,Zac Efron
12175,./dataset/imgs/Zac Efron/379.jpg,Zac Efron
12176,./dataset/imgs/Zac Efron/323.jpg,Zac Efron


In [11]:
imgs = []
classes = []
for i in dataset_df_test['image_pathname']:
    data = dataset_df.loc[dataset_df['image_pathname'] == './dataset/'+i]
    imgs.append(data.iat[0,0])
    classes.append(data.iat[0,1])
    
df_test = pd.DataFrame({
    'image_pathname': imgs,
    'class': classes
})

df_test

,image_pathname,class
0,./dataset/imgs/Adam Sandler/34.jpg,Adam Sandler
1,./dataset/imgs/Adam Sandler/236.jpg,Adam Sandler
2,./dataset/imgs/Adam Sandler/127.jpg,Adam Sandler
3,./dataset/imgs/Adam Sandler/111.jpg,Adam Sandler
4,./dataset/imgs/Adam Sandler/216.jpg,Adam Sandler
...,...,...
1655,./dataset/imgs/Zac Efron/240.jpg,Zac Efron
1656,./dataset/imgs/Zac Efron/68.jpg,Zac Efron
1657,./dataset/imgs/Zac Efron/124.jpg,Zac Efron
1658,./dataset/imgs/Zac Efron/283.jpg,Zac Efron


In [12]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(df_train['class'])

y_train = label_encoder.transform(df_train['class'])
y_test = label_encoder.transform(df_test['class'])

In [21]:
with tf.device('/CPU:0'):
    data_augmentation = tf.keras.Sequential([
         tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
         tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
         tf.keras.layers.experimental.preprocessing.RandomZoom(0.2)])

def augment(image, label):
  image = tf.cast(image, tf.float32)
  image = tf.image.resize(image, [IMG_SIZE, IMG_SIZE])
  image = (image / 255.0)
  image = tf.image.random_crop(image, size=[IMG_SIZE, IMG_SIZE, 3])
  image = tf.image.random_brightness(image, max_delta=0.5)
  return image, label

In [46]:
X_train =[]

for i in np.unique(df_train['class']):
    images = dataset_df.loc[dataset_df['class'] == i]['image_pathname']
    print(f'Loading images for class: {i}')
    X_class= []
    for img in images:
        image = cv2.imread(img)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        X_class.append(image)
    imgs.append(images)
    print(f'Starting data augmentation...')
    while len(X_class)<n_samples:
        index = np.random.choice(images)
        image = cv2.imread(img)
        image = tf.expand_dims(image, 0)
        augmented_image = data_augmentation(image)
        X_class.append(augmented_image[0])
    print(f'Data augmentation finished\n\n')
    X_train.extend(X_class)

Loading images for class: Adam Sandler
Starting data augmentation...
Data augmentation finished


Loading images for class: Alec Baldwin
Starting data augmentation...
Data augmentation finished


Loading images for class: Angelina Jolie
Starting data augmentation...
Data augmentation finished


Loading images for class: Anna Kournikova
Starting data augmentation...
Data augmentation finished


Loading images for class: Ashton Kutcher
Starting data augmentation...
Data augmentation finished


Loading images for class: Avril Lavigne
Starting data augmentation...
Data augmentation finished


Loading images for class: Barack Obama
Starting data augmentation...
Data augmentation finished


Loading images for class: Ben Affleck
Starting data augmentation...
Data augmentation finished


Loading images for class: Beyonce Knowles
Starting data augmentation...
Data augmentation finished


Loading images for class: Brad Pitt
Starting data augmentation...
Data augmentation finished


Loading image

Data augmentation finished




In [49]:
len(X_train)

30461

In [50]:
X_test =[]

for i in df_test.image_pathname:
    image = cv2.imread(i)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    X_test.append(image)

In [51]:
len(X_test)

1660

Concatenated list of Numbers : 
 
[1, 3, 5, 7, 9, 2, 4, 6, 8, 10]


In [42]:
dataset_df.loc[dataset_df['class'] == 'Adam Sandler']['image_pathname']

0      ./dataset/imgs/Adam Sandler/277.jpg
1      ./dataset/imgs/Adam Sandler/134.jpg
2      ./dataset/imgs/Adam Sandler/108.jpg
3        ./dataset/imgs/Adam Sandler/3.jpg
4       ./dataset/imgs/Adam Sandler/84.jpg
                      ...                 
103    ./dataset/imgs/Adam Sandler/208.jpg
104      ./dataset/imgs/Adam Sandler/7.jpg
105    ./dataset/imgs/Adam Sandler/111.jpg
106    ./dataset/imgs/Adam Sandler/152.jpg
107     ./dataset/imgs/Adam Sandler/46.jpg
Name: image_pathname, Length: 108, dtype: object

In [ ]:


class MyGenerator(Sequence):
    def __init__(self, image_pathnames, labels, batch_size, new_dims=(224, 224)):
        self.image_pathnames = image_pathnames
        self.labels = labels
        self.batch_size = batch_size
        self.new_dims = new_dims
    
    
    def __len__(self):        
        return math.ceil(len(self.image_pathnames) / float(self.batch_size))
    
    
    def __getitem__(self, idx):
        batch_begin = idx * self.batch_size
        batch_end = (idx + 1) * self.batch_size
        
        batch_image_pathnames = self.image_pathnames[batch_begin:batch_end]
        batch_labels = self.labels[batch_begin:batch_end]
        
        X_batch = self.__preprocess_batch(batch_image_pathnames)
        y_batch = np.array(batch_labels)
        
        return X_batch, y_batch
    
    
    def __preprocess_batch(self, batch_image_pathnames):
        # load the images as a feature matrix
        image_list = []  # list of numpy arrays
        
        for index, img_path in enumerate(batch_image_pathnames):
            img = cv2.imread(img_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, self.new_dims, interpolation=cv2.INTER_LINEAR)
            image_list.append(img)
        
        X = np.array(image_list)
        
        # feature scaling
        X = X / 255.0
        
        return X

# 2. Modelos

## 2.1 Criação dos modelos

## 2.2 Fine tuning

# 3. Validação